In [472]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
import re

In [69]:
# Returns Dataframe of document links for a given CIK
def get_sec_docs(cik="0001065088"):
    base_url = "https://www.sec.gov/cgi-bin/browse-edgar"
    inputted_cik = cik
    payload = {
        "action" : "getcompany",
        "CIK" : inputted_cik,
        "type" : "8-K",
        "output":"xml"
        #"dateb" : "20180331",
        #"count" : "100",
        #"owner" : "include"
    }
    sec_response = requests.get(url=base_url,params=payload)
    soup = BeautifulSoup(sec_response.text,'lxml')
    url_list = soup.findAll('filinghref')
    html_list = []
    # Get html version of links
    for link in url_list:
        link = link.string
        if link.split(".")[len(link.split("."))-1] == 'htm':
            txtlink = link + "l"
            html_list.append(txtlink)

    doc_list = []
    doc_name_list = []
    # Get links for txt versions of files
    for k in range(len(html_list)):
        txt_doc = html_list[k].replace("-index.html",".txt")
        doc_name = txt_doc.split("/")[-1]
        doc_list.append(txt_doc)
        doc_name_list.append(doc_name)
        # Create dataframe of CIK, doc name, and txt link
    df = pd.DataFrame(
        {
        "cik" : [cik]*len(html_list),
        "txt_link" : doc_list,
        "doc_name": doc_name_list
        }
    )
    return df

In [395]:
# Extracts text and submission datetime from document link
def extract_text(link):
    r = requests.get(link)
    #Parse 8-K document
    filing = BeautifulSoup(r.text,"html.parser")
    #Extract datetime
    submission_dt = filing.find("acceptance-datetime").string[:14]
    #Extract HTML sections
    for section in filing.findAll("html"):
        #Remove tables
        for table in section("table"):
            table.decompose()
        #Convert to unicode
        section = unicodedata.normalize("NFKD",section.text)
        section = section.replace("\t"," ").replace("\n"," ").replace("/s"," ")
    filing = "".join(str(section))
    
    return filing, submission_dt

In [314]:
ebay_df = get_sec_docs()

In [409]:
corpus = extract_text(ebay_df['txt_link'][1])

ConnectionError: HTTPConnectionPool(host='www.sec.gov', port=80): Max retries exceeded with url: /Archives/edgar/data/1065088/000106508818000004/0001065088-18-000004.txt (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x1158cfc18>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [253]:
r = requests.get(example)
filing = BeautifulSoup(r.text,"html.parser")

In [307]:
ebay_df['corpus'][1]

"    Document       UNITED STATES SECURITIES AND EXCHANGE COMMISSIONWashington, D.C. 20549FORM 8-KCURRENT REPORTPursuant to Section 13 or 15(d) ofthe Securities Exchange Act of 1934Date of Report (Date of earliest event reported): January 31, 2018 eBay Inc.(Exact name of registrant as specified in its charter)2025 Hamilton AvenueSan Jose, California 95125(Address of principal executive offices)(408) 376-7400(Registrant's telephone number, including area code)Not Applicable.(Former name or former address, if changed since last report.)Check the appropriate box below if the Form 8-K filing is intended to simultaneously satisfy the filing obligation of the registrant under any of the following provisions (see General Instruction A.2. below): ̈    Written communications pursuant to Rule 425 under the Securities Act (17 CFR 230.425) ̈    Soliciting material pursuant to Rule 14a-12 under the Exchange Act (17 CFR 240.14a-12) ̈    Pre-commencement communications pursuant to Rule 14d-2(b) under

In [408]:
corpus

'    Exhibit   Exhibit 99.1eBay Inc. Reports Fourth Quarter and Full Year 2017 ResultseBay Delivers Record Fourth Quarter Driven by Strong Holiday PerformanceSan Jose, California, January 31, 2018 - eBay Inc. (NASDAQ: EBAY), a global commerce leader, delivered revenue for the quarter ended December 31, 2017 of $2.6 billion, increasing 9% on an as-reported basis and 7% on a foreign exchange (FX) neutral basis, primarily driven by gross merchandise volume (GMV) of $24.4 billion, up 10% on an as-reported basis and 7% on an FX-Neutral basis. During the quarter, eBay delivered GAAP net loss from continuing operations of $2.6 billion, or $(2.51) per diluted share primarily driven by a $3.1 billion tax charge attributable to the enactment of the Tax Cuts and Jobs Act. Non-GAAP net income from continuing operations was $618 million, or $0.59 per diluted share. The company generated $988 million of operating cash flow and $796 million of free cash flow from continuing operations while also repu

In [485]:
def extract_text_file(link):
    with open(link) as f:
        r = f.read()
    #Parse each 8-K document
    parse_only = SoupStrainer("html")
    filing = BeautifulSoup(r,"html.parser",parse_only=parse_only)
    for table in filing("table"):
        table.decompose()
    filing = filing.get_text(strip=True)
    #submission_dt = filing.find("acceptance-datetime").string[:14]
    #Extract HTML sections
    
#     for section in filing.findAll("html"):
#         for table in section("table"):
#             table.decompose()
#         section = unicodedata.normalize("NFKD",section.text)
#         section = section.replace("\t"," ").replace("\n"," ").replace("/s"," ")
#     filing = "".join(str(section))
    #filing = " ".join(filing_sections)
    #filing = unicodedata.normalize("NFKD",filing.text)
    #filing = filing.replace("\t"," ").replace("\n"," ").replace("/s"," ")
    return filing

In [486]:
result = extract_text_file("8k.txt")
result

'DocumentUNITED STATES SECURITIES AND EXCHANGE COMMISSIONWashington,\xa0D.C. 20549FORM\xa08-KCURRENT REPORTPursuant to Section\xa013 or 15(d) ofthe Securities Exchange Act of 1934Date of Report (Date of earliest event reported):January\xa031, 2018eBay Inc.(Exact name of registrant as specified in its charter)2025 Hamilton AvenueSan\xa0Jose, California 95125(Address of principal executive offices)(408)\xa0376-7400(Registrant\'s telephone number, including area code)Not Applicable.(Former name or former address, if changed since last report.)Check the appropriate box below if the Form\xa08-K filing is intended to simultaneously satisfy the filing obligation of the registrant under any of the following provisions (see General Instruction\xa0A.2. below):¨Written communications pursuant to Rule\xa0425 under the Securities Act (17\xa0CFR 230.425)¨Soliciting material pursuant to Rule\xa014a-12 under the Exchange Act (17\xa0CFR 240.14a-12)¨Pre-commencement communications pursuant to Rule\xa014